## Processing NASA Logs  
  
This challenge was solved using Spark SQL to clean and wrangle the data and dataframes to answer the proposed questions.  
  
The challenge consists of answering the following questions:  
  
- Number of unique hosts  
- Total 404 errors  
- The 5 URLs that causes the most 404 errors  
- Number of 404 errors for each day  
- Total bytes returned  
  
Official dataset source https://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html
  
### Data:  
ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz  
ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz  
### About the dataset:  
The dataset holds all the HTTP requests to the server at NASA'S Kennedy Space Center during July and August of 1995.  